## compare stimulus parameters

In [1]:
# the following two lines indicate that external functions are auto-reloaded as soon as they change. 
%load_ext autoreload
%autoreload 2
# Print statements 
from __future__ import print_function # Python 2.x

In [2]:
# General stuff:
import sys
import argparse
import os
import json
import numpy as np
import math
import psycopg2
import cPickle
import numpy as np
import pandas as pd
from datetime import date
from tqdm import tqdm_notebook

# Plotting:
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
import matplotlib as mpl
%matplotlib inline

# External functions from subfolder /database_helpers. 
# as soon as you change something in there and press save, it will auto reload on next execution.
from database_helpers.psql_start import *
from database_helpers.create_tables import *
from database_helpers.write2tables import *
from postgres_analysis_helpers.general import *
from postgres_analysis_helpers.get_data import *

# register pickle type to retrieve binary data from database
psycopg2.extensions.register_type(psycopg2.extensions.new_type(psycopg2.BINARY.values, 'BINARY-PICKLE', cast_pickle))

Loaded analysis helpers: General
Loaded postgres_analysis_helpers -> general
Loaded postgres_analysis_helpers -> get_data


In [3]:
db_status = test_connect()
if db_status == False:
    print('Grrr... no database connection could be established.')
else:
    print('Yippiyeah! Database connection is established!')

Connecting to the PostgreSQL database...
Yippiyeah! Database connection is established!


### Make an empty dataframe and call the retrieval function

In [27]:
base_dataframe = pd.DataFrame() # create empty dataframe or feed in a base dataframe from before! 

In [28]:
psql = psql_neuroballs(base_dataframe)

61 animals across 8 experimenters found!
Total # of 5808 sessions (unique clusters: 11720)


In [29]:
psql.retrieve_data()

Retrieving data ... (Empty dataframe) ...
1045 entries retrieved.


In [30]:
dave = psql.data()

In [16]:
ragn = psql.data()

In [22]:
horst.head()

,tetrode_no,animal_id,n_drive_user,session_ts,cluster_no,session_name,stim_freq,stim_length,ibi
0,6,70644,horsto,2017-03-17 09:14:56,34,17032017laser4s1,10.101010,1.000000,2098.0
1,5,71462,horsto,2017-04-10 10:06:32,5,10042017laser4(40mW)s1,20.408163,2.000000,2047.0
2,7,71461,horsto,2017-04-10 19:33:45,17,10042017laser3(50mW)s2,20.094454,0.995745,2048.0
3,5,71710,horsto,2017-04-11 12:14:46,35,11042017laser2(40mW)s1,10.101010,2.000000,2097.0
4,7,71461,horsto,2017-04-12 16:10:09,24,12042017laser1(40mW)s1,10.101010,1.000000,2098.0


In [23]:
ragn.head()

,tetrode_no,animal_id,n_drive_user,session_ts,cluster_no,session_name,stim_freq,stim_length,ibi
0,1,70375,ragnihiij,2017-03-15 09:32:48,6,Mouse70375_20170315_01_5Hz20mW,5.0,1.0,2200.0
1,1,70375,ragnihiij,2017-03-15 09:32:48,6,Mouse70375_20170315_01_10Hz20mW,10.0,1.0,2100.0
2,2,70375,ragnihiij,2017-03-15 09:32:48,19,Mouse70375_20170315_01_5Hz20mW,5.0,1.0,2200.0
3,2,70375,ragnihiij,2017-03-15 09:32:48,19,Mouse70375_20170315_01_10Hz20mW,10.0,1.0,2100.0
4,2,70375,ragnihiij,2017-03-15 09:32:48,5,Mouse70375_20170315_01_5Hz20mW,5.0,1.0,2200.0


In [31]:
dave.head()

,tetrode_no,animal_id,n_drive_user,session_ts,cluster_no,session_name,stim_freq,stim_length,ibi
0,8,59394,davidcr,2017-04-25 15:15:01,43,25042017laserSess1(60mw),10.00000,1.0,2097.000000
1,5,59392,davidcr,2017-04-27 13:36:23,5,27042017laserSess2(80mw),10.00000,1.0,2097.090909
2,6,72257,davidcr,2017-04-15 14:55:58,6,15042017laserSess2(80mw),9.90099,2.0,2097.000000
3,6,72257,davidcr,2017-04-15 14:55:58,6,15042017laserSess2(40mw),9.90099,2.0,2097.000000
4,6,72257,davidcr,2017-04-15 14:55:58,6,15042017laserSess1(80mw),10.00000,1.0,2097.000000


In [32]:
ragn.stim_freq.values

array([  5.        ,  10.        ,   5.        , ...,  10.00333444,
        10.        ,  10.        ])

In [40]:
ten_ragn = ragn.stim_freq.values[(ragn.stim_freq.values > 8.0) &(ragn.stim_freq.values < 12.0)]

In [41]:
print(np.std(ten_ragn))

0.00129363571299


In [42]:
ten_horst = horst.stim_freq.values[(horst.stim_freq.values > 8.0) &(horst.stim_freq.values < 12.0)]

In [43]:
print(np.std(ten_horst))

0.0232935750124


In [44]:
ten_dave = dave.stim_freq.values[(dave.stim_freq.values > 8.0) &(dave.stim_freq.values < 12.0)]

In [45]:
print(np.std(ten_dave))

0.0838253325455


In [58]:
#for feat in ['stim_freq','stim_length','ibi']:
for nuser in [ragn,horst,dave]:
        stim_freq_10 = nuser['stim_freq'].values[(nuser['stim_freq'].values > 8.0) & (nuser['stim_freq'].values < 12.0)]
        stim_freq_10_ptp = np.ptp(stim_freq_10)
        
        stim_len_1 = nuser['stim_length'].values[(nuser['stim_length'].values > .8) & (nuser['stim_length'].values < 1.2)]
        stim_len_1_ptp = np.ptp(stim_len_1)

        #stim_len_2 = nuser['stim_length'].values[(nuser['stim_length'].values > 1.8) & (nuser['stim_length'].values < 2.2)]
        #stim_len_2_ptp = np.ptp(stim_len_2)

        ibi = nuser['ibi'].values[(nuser['ibi'].values > 1800) & (nuser['ibi'].values < 2200)]
        ibi_ptp = np.ptp(ibi)
        
        print('Freq (10 Hz): {} | Stim Length (1ms) {} | IBI (2000) {}'.format(stim_freq_10_ptp,stim_len_1_ptp,ibi_ptp))

Freq (10 Hz): 0.0047123645 | Stim Length (1ms) 0.039855072464 | IBI (2000) 0.0
Freq (10 Hz): 0.101010101 | Stim Length (1ms) 0.076086956522 | IBI (2000) 93.78947368
Freq (10 Hz): 0.20460605789 | Stim Length (1ms) 0.028985507246 | IBI (2000) 130.58139535
